In [1]:
import os
import cv2
from glob import glob
import pandas as pd
import random
import numpy
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from PIL import Image
import PIL
import plotly.express as px
from sklearn.model_selection import train_test_split
from tqdm import tqdm 
%matplotlib inline

In [2]:
TRAIN_PATH = './input/kidf-kpop-idol-dataset-female/HQ_512x512/HQ_512x512/'
TEST_PATH = './input/kidf-kpop-idol-dataset-female/test_final_with_degrad/test/'

In [3]:
train = pd.read_csv('./input/kidf-kpop-idol-dataset-female/kid_f_train.csv')
test = pd.read_csv('./input/kidf-kpop-idol-dataset-female/kid_f_test.csv')

In [4]:
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

TRAIN_PATH = './data/train/'
VAL_PATH = './data/val/'

# TRAIN_PATH = './input/kidf-kpop-idol-dataset-female/HQ_512x512/HQ_512x512/'
# Transform 정의
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)), transforms.Resize(256), transforms.FiveCrop(224),
     transforms.Lambda(lambda crops: torch.stack([(crop) for crop in crops]))])

# def npy_loader(path):
#     sample = torchvision.io.read_image(path)
#     return sample


# CIFAR10 TEST 데이터 정의
# test_ds = datasets.DatasetFolder(root=TEST_PATH, loader=npy_loader,transform=transform, extensions=['.jpg'],target_transform=None)

# class_names = train['name'].unique().tolist()
# tclass_names = test['name'].unique().tolist()
# available_class = list(set(class_names)&set(tclass_names))
# print(len(available_class))

# classes = available_class

In [5]:
train_ds = datasets.ImageFolder(root=TRAIN_PATH,  transform=transform, target_transform=None)
val_ds = datasets.ImageFolder(root=VAL_PATH,  transform=transform, target_transform=None)

In [6]:
train_dl=DataLoader(train_ds, batch_size=2, shuffle=True)
val_dl=DataLoader(val_ds, batch_size=2, shuffle=False)

In [7]:
class_names = train['name'].unique().tolist()
tclass_names = test['name'].unique().tolist()
available_class = list(set(class_names)&set(tclass_names))
print(len(available_class))
classes = (available_class)

23


In [8]:
# for i,rows in full_df.iterrows():
#     cat = rows['name']
#     ipath = TRAIN_PATH + rows['file_name']
#     initpath=ipath[0:-4]
#     section = rows['type']
#     opath = ipath.replace(f"./input/kidf-kpop-idol-dataset-female/HQ_512x512/HQ_512x512/",f"./data/{section}/{cat}/")
#     os.rename(ipath, opath)    

In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary
from torch import optim
from torch.optim.lr_scheduler import StepLR

# dataset and transformation
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import os

# display images
from torchvision import utils
import matplotlib.pyplot as plt
%matplotlib inline

# utils
import numpy as np
from torchsummary import summary
import time
import copy

In [10]:
VGG_types = {
    'VGG11' : [64, 'M', 128, 'M', 256, 256, 'M', 512,512, 'M',512,512,'M'],
    'VGG13' : [64,64, 'M', 128, 128, 'M', 256, 256, 'M', 512,512, 'M', 512,512,'M'],
    'VGG16' : [64,64, 'M', 128, 128, 'M', 256, 256,256, 'M', 512,512,512, 'M',512,512,512,'M'],
    'VGG19' : [64,64, 'M', 128, 128, 'M', 256, 256,256,256, 'M', 512,512,512,512, 'M',512,512,512,512,'M']
}

In [12]:
class VGGnet(nn.Module):
    def __init__(self, model, in_channels=3, num_classes=23, init_weights=True):
        super(VGGnet,self).__init__()
        self.in_channels = in_channels

        # create conv_layers corresponding to VGG type
        self.conv_layers = self.create_conv_laters(VGG_types[model])

        self.fcs = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(4096, num_classes),
        )

        # weight initialization
        if init_weights:
            self._initialize_weights()

    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(-1, 512 * 7 * 7)
        x = self.fcs(x)
        return x

    # defint weight initialization function
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)
    
    # define a function to create conv layer taken the key of VGG_type dict 
    def create_conv_laters(self, architecture):
        layers = []
        in_channels = self.in_channels # 3

        for x in architecture:
            if type(x) == int: # int means conv layer
                out_channels = x

                layers += [nn.Conv2d(in_channels=in_channels, out_channels=out_channels,
                                     kernel_size=(3,3), stride=(1,1), padding=(1,1)),
                           nn.BatchNorm2d(x),
                           nn.ReLU()]
                in_channels = x
            elif x == 'M':
                layers += [nn.MaxPool2d(kernel_size=(2,2), stride=(2,2))]
        
        return nn.Sequential(*layers)

# define device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

# creat VGGnet object
model = VGGnet('VGG19', in_channels=3, num_classes=23, init_weights=True).to(device)
print(model)

cuda
VGGnet(
  (conv_layers): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU()
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU()
    (13): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256, kernel_size=(3,

In [13]:
summary(model, input_size=(3, 224, 224), device=device.type)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
       BatchNorm2d-2         [-1, 64, 224, 224]             128
              ReLU-3         [-1, 64, 224, 224]               0
            Conv2d-4         [-1, 64, 224, 224]          36,928
       BatchNorm2d-5         [-1, 64, 224, 224]             128
              ReLU-6         [-1, 64, 224, 224]               0
         MaxPool2d-7         [-1, 64, 112, 112]               0
            Conv2d-8        [-1, 128, 112, 112]          73,856
       BatchNorm2d-9        [-1, 128, 112, 112]             256
             ReLU-10        [-1, 128, 112, 112]               0
           Conv2d-11        [-1, 128, 112, 112]         147,584
      BatchNorm2d-12        [-1, 128, 112, 112]             256
             ReLU-13        [-1, 128, 112, 112]               0
        MaxPool2d-14          [-1, 128,

In [14]:
loss_func = nn.CrossEntropyLoss(reduction="sum")
opt = optim.Adam(model.parameters(), lr=0.01)

# get learning rate 
def get_lr(opt):
    for param_group in opt.param_groups:
        return param_group['lr']

current_lr = get_lr(opt)
print('current lr={}'.format(current_lr))

# define learning rate scheduler
# from torch.optim.lr_scheduler import CosineAnnealingLR
# lr_scheduler = CosineAnnealingLR(opt, T_max=2, eta_min=1e-5)

from torch.optim.lr_scheduler import StepLR
lr_scheduler = StepLR(opt, step_size=30, gamma=0.1)

current lr=0.01


In [15]:
def metrics_batch(output, target):
    # get output class
    pred = output.argmax(dim=1, keepdim=True)
    
    # compare output class with target class
    corrects=pred.eq(target.view_as(pred)).sum().item()
    return corrects

def loss_batch(loss_func, output, target, opt=None):
    
    # get loss 
    loss = loss_func(output, target)
    
    # get performance metric
    metric_b = metrics_batch(output,target)
    
    if opt is not None:
        opt.zero_grad()
        loss.backward()
        opt.step()

    return loss.item(), metric_b

def loss_epoch(model,loss_func,dataset_dl,sanity_check=False,opt=None):
    running_loss=0.0
    running_metric=0.0
    len_data=len(dataset_dl.dataset)

    for xb, yb in dataset_dl:
        # move batch to device
        print(yb)
        xb=xb.to(device)
        yb=yb.to(device)
        
        # Five crop : bs, crops, chnnel, h, w
        # making dimmension (bs, c, h, w)
        bs, ncrops, c, h, w = xb.size()
        output_=model(xb.view(-1, c, h, w))
        output = output_.view(bs, ncrops, -1).mean(1)
        
        # get loss per batch
        loss_b,metric_b=loss_batch(loss_func, output, yb, opt)
        
        # update running loss
        running_loss+=loss_b
        
        # update running metric
        if metric_b is not None:
            running_metric+=metric_b

        # break the loop in case of sanity check
        if sanity_check is True:
            break
    
    # average loss value
    loss=running_loss/float(len_data)
    
    # average metric value
    metric=running_metric/float(len_data)
    
    return loss, metric

In [16]:
def train_val(model, params):
    # extract model parameters
    num_epochs=params["num_epochs"]
    loss_func=params["loss_func"]
    opt=params["optimizer"]
    train_dl=params["train_dl"]
    val_dl=params["val_dl"]
    sanity_check=params["sanity_check"]
    lr_scheduler=params["lr_scheduler"]
    path2weights=params["path2weights"]
    
    # history of loss values in each epoch
    loss_history={
        "train": [],
        "val": [],
    }
    
    # histroy of metric values in each epoch
    metric_history={
        "train": [],
        "val": [],
    }
    
    # 가중치를 저장할 때, 코랩 GPU 오류나서 생략했습니다.
    # a deep copy of weights for the best performing model
    # best_model_wts = copy.deepcopy(model.state_dict())
    
    # initialize best loss to a large value
    best_loss=float('inf')
    
    # main loop
    for epoch in range(num_epochs):
        # check 1 epoch start time
        start_time = time.time()

        # get current learning rate
        current_lr=get_lr(opt)
        print('Epoch {}/{}, current lr={}'.format(epoch, num_epochs - 1, current_lr))
        
        # train model on training dataset
        model.train()
        train_loss, train_metric=loss_epoch(model,loss_func,train_dl,sanity_check,opt)

        # collect loss and metric for training dataset
        loss_history["train"].append(train_loss)
        metric_history["train"].append(train_metric)
        
        # evaluate model on validation dataset    
        model.eval()
        with torch.no_grad():
            val_loss, val_metric=loss_epoch(model,loss_func,val_dl,sanity_check)
        
       
        # store best model
        if val_loss < best_loss:
            best_loss = val_loss
            best_model_wts = copy.deepcopy(model.state_dict())
            
            # # store weights into a local file
            # torch.save(model.state_dict(), path2weights)
            # print("Copied best model weights!")
        
        # collect loss and metric for validation dataset
        loss_history["val"].append(val_loss)
        metric_history["val"].append(val_metric)
        
        # learning rate schedule
        lr_scheduler.step()

        print("train loss: %.6f, dev loss: %.6f, accuracy: %.2f, time: %.4f s" %(train_loss,val_loss,100*val_metric, time.time()-start_time))
        print("-"*10) 

    ## load best model weights
    # model.load_state_dict(best_model_wts)
        
    return model, loss_history, metric_history

In [17]:
params_train = {
    'num_epochs':100,
    'optimizer':opt,
    'loss_func':loss_func,
    'train_dl':train_dl,
    'val_dl':val_dl,
    'sanity_check':False,
    'lr_scheduler':lr_scheduler,
    'path2weights':'./models/weights.pt',
}

# create the directory that stores weights.pt
def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSerror:
        print('Error')
createFolder('./models')

In [18]:
model, loss_hist, metric_hist = train_val(model, params_train)

Epoch 0/99, current lr=0.01
tensor([15, 15])
tensor([8, 9])
tensor([8, 8])
tensor([15, 15])
tensor([18,  8])
tensor([15,  6])
tensor([15, 18])
tensor([18, 15])
tensor([7, 7])
tensor([21,  1])
tensor([14, 15])
tensor([22,  3])
tensor([ 7, 18])
tensor([8, 3])
tensor([ 8, 15])
tensor([8, 8])
tensor([ 8, 18])
tensor([9, 8])
tensor([18,  7])
tensor([10,  8])
tensor([8, 6])
tensor([18, 15])
tensor([15,  8])
tensor([7, 7])
tensor([ 8, 15])
tensor([17, 15])
tensor([ 8, 15])
tensor([ 8, 18])
tensor([18, 18])
tensor([ 9, 15])
tensor([15, 10])
tensor([18, 18])
tensor([17,  7])
tensor([15, 18])
tensor([15, 18])
tensor([11,  6])
tensor([15,  8])
tensor([15,  8])
tensor([15, 16])
tensor([15, 18])
tensor([17, 15])
tensor([15, 13])
tensor([14,  3])
tensor([8, 2])
tensor([ 3, 18])
tensor([15, 18])
tensor([ 5, 11])
tensor([18,  7])
tensor([18,  5])
tensor([18, 15])
tensor([18, 15])
tensor([ 8, 15])
tensor([ 8, 15])
tensor([5, 8])
tensor([ 9, 15])
tensor([14, 14])
tensor([18,  9])
tensor([18, 18])
tensor

tensor([15, 18])
tensor([18, 18])
tensor([15,  3])
tensor([15, 15])
tensor([17, 18])
tensor([18, 18])
tensor([18, 15])
tensor([15, 18])
tensor([18,  8])
tensor([15, 15])
tensor([15,  8])
tensor([18, 18])
tensor([ 9, 15])
tensor([ 9, 18])
tensor([8, 8])
tensor([ 5, 15])
tensor([15,  8])
tensor([18,  8])
tensor([8, 9])
tensor([18,  8])
tensor([15,  8])
tensor([8, 7])
tensor([18, 14])
tensor([ 8, 15])
tensor([18, 18])
tensor([8, 8])
tensor([15, 18])
tensor([15,  3])
tensor([18, 15])
tensor([18, 18])
tensor([7, 7])
tensor([15,  7])
tensor([15, 18])
tensor([7, 8])
tensor([9, 8])
tensor([ 8, 15])
tensor([19,  8])
tensor([17, 18])
tensor([7, 8])
tensor([10, 16])
tensor([16, 18])
tensor([ 8, 18])
tensor([18, 15])
tensor([15, 15])
tensor([15, 18])
tensor([ 8, 18])
tensor([15,  8])
tensor([ 8, 18])
tensor([ 9, 14])
tensor([ 8, 18])
tensor([ 8, 14])
tensor([15, 15])
tensor([13, 18])
tensor([ 3, 15])
tensor([18,  8])
tensor([ 8, 15])
tensor([15, 14])
tensor([18, 14])
tensor([18,  7])
tensor([13, 1

tensor([ 8, 18])
tensor([15,  7])
tensor([18, 15])
tensor([ 3, 18])
tensor([18,  8])
tensor([15,  8])
tensor([19, 18])
tensor([8, 0])
tensor([22, 15])
tensor([18, 18])
tensor([15,  8])
tensor([18, 18])
tensor([18, 18])
tensor([18,  8])
tensor([8, 7])
tensor([8, 8])
tensor([19, 15])
tensor([ 7, 20])
tensor([18,  8])
tensor([15, 14])
tensor([14, 18])
tensor([18,  9])
tensor([15, 15])
tensor([8, 8])
tensor([15, 15])
tensor([13, 15])
tensor([18, 15])
tensor([ 8, 15])
tensor([18, 18])
tensor([ 8, 15])
tensor([15,  8])
tensor([18, 22])
tensor([15,  8])
tensor([18, 18])
tensor([14,  8])
tensor([ 5, 15])
tensor([10, 18])
tensor([8, 3])
tensor([15,  8])
tensor([21, 18])
tensor([ 9, 20])
tensor([8, 9])
tensor([ 6, 15])
tensor([ 8, 18])
tensor([ 2, 15])
tensor([7, 8])
tensor([22, 15])
tensor([18,  5])
tensor([8, 7])
tensor([18, 19])
tensor([18, 13])
tensor([15,  8])
tensor([ 7, 14])
tensor([15,  7])
tensor([8, 7])
tensor([13, 15])
tensor([ 7, 15])
tensor([15,  5])
tensor([15,  8])
tensor([ 9, 14]

tensor([15,  8])
tensor([ 8, 15])
tensor([18, 18])
tensor([14, 18])
tensor([ 8, 15])
tensor([15,  9])
tensor([7, 6])
tensor([15,  9])
tensor([18, 15])
tensor([18, 15])
tensor([18, 15])
tensor([17, 14])
tensor([21, 14])
tensor([10,  8])
tensor([ 3, 15])
tensor([18,  5])
tensor([14, 13])
tensor([18,  7])
tensor([ 5, 18])
tensor([ 5, 15])
tensor([ 3, 15])
tensor([ 8, 15])
tensor([19,  9])
tensor([15, 15])
tensor([18, 18])
tensor([ 8, 16])
tensor([18, 14])
tensor([18, 18])
tensor([ 8, 18])
tensor([ 2, 18])
tensor([ 7, 18])
tensor([18,  7])
tensor([ 3, 15])
tensor([15,  8])
tensor([ 8, 18])
tensor([18, 15])
tensor([ 7, 14])
tensor([ 9, 15])
tensor([12,  7])
tensor([ 9, 18])
tensor([ 8, 10])
tensor([15,  8])
tensor([7, 7])
tensor([ 8, 14])
tensor([ 8, 14])
tensor([14,  8])
tensor([ 8, 18])
tensor([15, 18])
tensor([ 7, 18])
tensor([15, 15])
tensor([10, 18])
tensor([ 8, 15])
tensor([8, 8])
tensor([ 8, 15])
tensor([18, 13])
tensor([17, 18])
tensor([15,  9])
tensor([15, 15])
tensor([ 5, 14])
ten

tensor([15, 18])
tensor([13, 18])
tensor([18, 18])
tensor([8, 8])
tensor([18, 15])
tensor([19, 15])
tensor([15, 18])
tensor([18,  8])
tensor([18,  9])
tensor([7, 3])
tensor([ 8, 15])
tensor([18,  5])
tensor([18,  3])
tensor([7, 7])
tensor([ 4, 16])
tensor([9, 9])
tensor([8, 8])
tensor([19,  3])
tensor([17, 15])
tensor([8, 8])
tensor([18, 15])
tensor([18, 18])
tensor([ 8, 18])
tensor([3, 8])
tensor([ 7, 18])
tensor([8, 8])
tensor([18, 18])
tensor([19, 15])
tensor([15, 15])
tensor([15,  7])
tensor([ 8, 15])
tensor([15, 13])
tensor([18,  9])
tensor([7, 3])
tensor([15,  8])
tensor([5, 1])
tensor([ 7, 15])
tensor([15, 15])
tensor([ 8, 19])
tensor([ 5, 21])
tensor([8, 8])
tensor([15,  3])
tensor([ 5, 15])
tensor([ 7, 16])
tensor([15, 15])
tensor([15,  8])
tensor([8, 8])
tensor([ 9, 18])
tensor([18, 17])
tensor([18, 18])
tensor([18, 18])
tensor([15,  8])
tensor([3, 7])
tensor([ 8, 19])
tensor([18,  8])
tensor([15, 15])
tensor([14, 15])
tensor([ 8, 15])
tensor([9, 7])
tensor([18,  8])
tensor([

...

tensor([ 5, 18])
tensor([18, 16])
tensor([15, 16])
tensor([16, 15])
tensor([14,  8])
tensor([8, 7])
tensor([18,  2])
tensor([22, 10])
tensor([ 8, 15])
tensor([15,  8])
tensor([15,  8])
tensor([15, 14])
tensor([18, 14])
tensor([18, 15])
tensor([15, 15])
tensor([18, 13])
tensor([15,  5])
tensor([8, 3])
tensor([8, 8])
tensor([15,  9])
tensor([15, 18])
tensor([18, 15])
tensor([18, 18])
tensor([15, 20])
tensor([ 8, 15])
tensor([15, 15])
tensor([ 5, 15])
tensor([18, 15])
tensor([1, 8])
tensor([9, 3])
tensor([18,  8])
tensor([ 7, 14])
tensor([15, 15])
tensor([17,  8])
tensor([ 8, 18])
tensor([18,  2])
tensor([22,  8])
tensor([15, 15])
tensor([18, 13])
tensor([ 9, 15])
tensor([18, 15])
tensor([15, 15])
tensor([7, 8])
tensor([18,  0])
tensor([18, 15])
tensor([18,  8])
tensor([14, 15])
tensor([19,  9])
tensor([18, 15])
tensor([18,  3])
tensor([15, 15])
tensor([9, 8])
tensor([ 8, 18])
tensor([18,  8])
tensor([18, 18])
tensor([ 8, 15])
tensor([ 7, 15])
tensor([18,  7])
tensor([15, 15])
tensor([8, 

tensor([15, 15])
tensor([15, 15])
tensor([15, 15])
tensor([15, 15])
tensor([15, 15])
tensor([15, 15])
tensor([15, 15])
tensor([15, 15])
tensor([15, 15])
tensor([15, 15])
tensor([15, 15])
tensor([15, 15])
tensor([15, 15])
tensor([15, 15])
tensor([15, 15])
tensor([15, 15])
tensor([15, 15])
tensor([15, 15])
tensor([15, 15])
tensor([15, 15])
tensor([15, 15])
tensor([15, 15])
tensor([15, 15])
tensor([15, 15])
tensor([15, 15])
tensor([15, 15])
tensor([15, 15])
tensor([15, 15])
tensor([15, 15])
tensor([15, 15])
tensor([15, 15])
tensor([15, 15])
tensor([15, 15])
tensor([15, 15])
tensor([15, 15])
tensor([15, 15])
tensor([15, 15])
tensor([15, 15])
tensor([15, 15])
tensor([15, 15])
tensor([15, 15])
tensor([15, 15])
tensor([15, 15])
tensor([15, 15])
tensor([15, 15])
tensor([15, 15])
tensor([15, 15])
tensor([15, 15])
tensor([15, 15])
tensor([15, 15])
tensor([15, 15])
tensor([15, 15])
tensor([15, 15])
tensor([15, 15])
tensor([15, 15])
tensor([15, 15])
tensor([15, 15])
tensor([15, 15])
tensor([15, 15

tensor([5, 8])
tensor([6, 8])
tensor([ 8, 15])
tensor([19, 18])
tensor([ 8, 13])
tensor([5, 8])
tensor([4, 9])
tensor([7, 8])
tensor([8, 9])
tensor([8, 8])
tensor([ 8, 15])
tensor([8, 8])
tensor([15, 18])
tensor([14,  8])
tensor([18, 15])
tensor([18, 15])
tensor([14, 15])
tensor([15,  5])
tensor([13,  9])
tensor([14,  5])
tensor([18,  7])
tensor([17,  8])
tensor([18,  9])
tensor([18, 21])
tensor([ 9, 18])
tensor([ 7, 18])
tensor([18,  8])
tensor([18, 18])
tensor([12, 18])
tensor([ 8, 18])
tensor([7, 8])
tensor([ 3, 15])
tensor([18,  8])
tensor([15,  8])
tensor([15, 18])
tensor([8, 8])
tensor([19, 18])
tensor([14, 15])
tensor([15, 18])
tensor([15,  7])
tensor([7, 7])
tensor([15, 18])
tensor([ 8, 16])
tensor([19, 14])
tensor([15,  8])
tensor([ 7, 15])
tensor([18,  7])
tensor([18,  7])
tensor([15,  8])
tensor([14,  8])
tensor([ 8, 18])
tensor([7, 8])
tensor([10,  8])
tensor([ 8, 14])
tensor([15, 13])
tensor([ 9, 15])
tensor([15, 18])
tensor([15,  7])
tensor([18,  8])
tensor([8, 9])
tensor

tensor([ 8, 18])
tensor([7, 8])
tensor([15,  8])
tensor([18, 18])
tensor([15,  9])
tensor([15,  7])
tensor([18, 18])
tensor([ 7, 22])
tensor([8, 8])
tensor([15, 15])
tensor([18,  9])
tensor([14,  8])
tensor([18,  7])
tensor([15, 15])
tensor([18,  8])
tensor([15, 16])
tensor([ 8, 18])
tensor([8, 8])
tensor([18,  9])
tensor([15, 15])
tensor([18, 15])
tensor([18,  8])
tensor([8, 7])
tensor([15, 18])
tensor([18,  8])
tensor([15, 15])
tensor([13, 15])
tensor([15,  8])
tensor([ 7, 15])
tensor([ 7, 15])
tensor([8, 9])
tensor([15, 15])
tensor([ 6, 14])
tensor([5, 7])
tensor([15, 18])
tensor([19, 18])
tensor([20, 15])
tensor([7, 5])
tensor([13, 15])
tensor([18, 15])
tensor([5, 9])
tensor([8, 8])
tensor([ 8, 18])
tensor([14, 18])
tensor([15,  5])
tensor([18, 18])
tensor([18, 15])
tensor([15, 18])
tensor([18, 18])
tensor([15, 18])
tensor([ 8, 18])
tensor([13,  8])
tensor([9, 7])
tensor([15, 15])
tensor([18,  3])
tensor([15,  5])
tensor([18, 12])
tensor([ 9, 11])
tensor([7, 5])
tensor([8, 7])
tens

KeyboardInterrupt: 

In [88]:
sanity_check=False

In [ ]:
model